# train_hmm.ipynb

In [1]:
from pandas import DataFrame, read_csv
import pandas as pd
import numpy as np
import sys, os

from pomegranate import *


In [2]:
dir_path = '/Users/Felix/GitHub/HMMicro/hmm_scripts'

In [3]:
# Get 2-state transition matrix
trans_mat_path = os.path.join(dir_path, 'trans_mat.tsv')
trans_mat = read_csv(trans_mat_path, sep='\t', header=0, index_col=None)
trans_mat.index = state_nms = trans_mat.columns.tolist() # Get state names as list
state_nms = np.array(state_nms)

# Get emission parameters (mean, sd)
emit_params_path = os.path.join(dir_path, 'emit_params.tsv')
emit_params = read_csv(emit_params_path, sep='\t', header=0, index_col=None)

In [4]:
trans_mat

,stateA,stateB
stateA,0.7,0.3
stateB,0.2,0.8


In [5]:
emit_params

,stateA,stateB
0,5,1
1,1,4


In [6]:
# Get simulated sequence
state_seq_path = os.path.join(dir_path, 'state_seq.tsv')
sim_state_seq = read_csv(state_seq_path, sep='\t', header=None, index_col=None)
sim_state_seq = sim_state_seq.iloc[:,0].tolist()

# Get simulated sequence
emit_seq_path = os.path.join(dir_path, 'emit_seq.tsv')
sim_emit_seq = read_csv(emit_seq_path, sep='\t', header=None, index_col=None)
sim_emit_seq = sim_emit_seq.iloc[:,0].tolist()

In [7]:
# Construct HMM
s1 = State( NormalDistribution(3,3), name=str(state_nms[0]) )
s2 = State( NormalDistribution(3,3), name=str(state_nms[1]) )

model = HiddenMarkovModel()
model.add_states( [s1, s2] )
model.add_transition( model.start, s1, 1)
model.add_transition( model.start, s2, 0)
model.add_transition( s1, s1, 0.5)
model.add_transition( s2, s2, 0.5)
model.add_transition( s1, s2, 0.5)
model.add_transition( s2, s1, 0.5)
model.bake()

In [21]:
# Get the labels for labeled training
use_labels = []
for i in sim_state_seq:
    if i == state_nms[0]:
        use_labels.append(0)
    if i == state_nms[1]:
        use_labels.append(1)
use_labels = [np.array(use_labels)]

In [26]:
print len( [sim_state_seq] )
print len( [np.array(sim_emit_seq)] )

1
1


In [24]:
# Fit HMM
model.fit([np.array(sim_emit_seq)], distribution_inertia=0.1, edge_inertia=0.1,
          labels=[sim_state_seq], algorithm='labeled')

Training improvement: 0.0
Total Training Improvement: 0.0


Exception ValueError: "'stateA' is not in list" in 'pomegranate.hmm.HiddenMarkovModel._labeled_summarize' ignored
Exception ValueError: "'stateA' is not in list" in 'pomegranate.hmm.HiddenMarkovModel._labeled_summarize' ignored


0.0

In [9]:
model.dense_transition_matrix()

array([[ 0.7059275 ,  0.2940725 ,  0.        ,  0.        ],
       [ 0.17954023,  0.82045977,  0.        ,  0.        ],
       [ 1.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ]])

In [11]:
for st in model.states:
    if st.name in state_nms:
        print(st.name)
        print(st.distribution)

stateA
{
    "frozen" :false,
    "parameters" :[
        5.039671116842356,
        0.940528007442133
    ],
    "class" :"Distribution",
    "name" :"NormalDistribution"
}
stateB
{
    "frozen" :false,
    "parameters" :[
        1.1501819222924201,
        4.056530371160682
    ],
    "class" :"Distribution",
    "name" :"NormalDistribution"
}


In [54]:
model._labeled_summarize()

TypeError: _labeled_summarize() takes exactly 3 positional arguments (0 given)